# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('ClientsData.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [4]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score

In [11]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(Xtrain, ytrain)

ypred = knn.predict(Xtest)

print("Матрица ошибок:\n", confusion_matrix(ytest, ypred))
print("\nОтчёт по классификации:\n", classification_report(ytest, ypred, digits=3))


Матрица ошибок:
 [[3982   26]
 [ 553    6]]

Отчёт по классификации:
               precision    recall  f1-score   support

           0      0.878     0.994     0.932      4008
           1      0.188     0.011     0.020       559

    accuracy                          0.873      4567
   macro avg      0.533     0.502     0.476      4567
weighted avg      0.794     0.873     0.821      4567



TP (True Positive) — модель правильно предсказала положительный класс.

TN (True Negative) — модель правильно предсказала отрицательный класс.

Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(Xtrain, ytrain)

ypred = logreg.predict(Xtest)


print("Матрица ошибок:\n", confusion_matrix(ytest, ypred))
print("\nОтчёт по классификации:\n", classification_report(ytest, ypred, digits=3))

Матрица ошибок:
 [[4004    4]
 [ 559    0]]

Отчёт по классификации:
               precision    recall  f1-score   support

           0      0.877     0.999     0.934      4008
           1      0.000     0.000     0.000       559

    accuracy                          0.877      4567
   macro avg      0.439     0.500     0.467      4567
weighted avg      0.770     0.877     0.820      4567



Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [19]:
# Предсказания меток
ypred = logreg.predict(Xtest)

# Оценка точности
print("Accuracy:", accuracy_score(ytest, ypred))

# Матрица ошибок и отчёт
print("Матрица ошибок:\n", confusion_matrix(ytest, ypred))
print("\nОтчёт по классификации:\n", classification_report(ytest, ypred, digits=3))

# Дополнительно: предсказание вероятностей
yproba = logreg.predict_proba(Xtest)

# Посмотрим вероятности положительного класса (1) для первых 10 примеров
print("\nВероятности положительного класса (1) для первых 10 объектов:")
print(yproba[:10, 1])

Accuracy: 0.8767243266914824
Матрица ошибок:
 [[4004    4]
 [ 559    0]]

Отчёт по классификации:
               precision    recall  f1-score   support

           0      0.877     0.999     0.934      4008
           1      0.000     0.000     0.000       559

    accuracy                          0.877      4567
   macro avg      0.439     0.500     0.467      4567
weighted avg      0.770     0.877     0.820      4567


Вероятности положительного класса (1) для первых 10 объектов:
[0.07132911 0.11530907 0.29188183 0.21909763 0.13873437 0.04492519
 0.06424646 0.03588642 0.05715797 0.06871201]


In [ ]:
Логистическая регрессия предсказывает вероятности, и по умолчанию использует порог 0.5
Из-за дисбаланса большинство вероятностей класса 1 оказываются ниже 0.5, поэтому модель классифицирует почти всё как класс 0.

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [20]:
# Предсказание вероятностей для валидационной выборки
probs_val = logreg.predict_proba(Xval)[:, 1]

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    preds = (probs_val >= threshold).astype(int)
    
    precision = precision_score(yval, preds)
    recall = recall_score(yval, preds)
    
    if precision >= 0.13 and recall > max_recall:
        max_recall = recall
        thr = threshold
        prec = precision

print(f"Оптимальный порог: {thr:.3f}")
print(f"Recall: {max_recall:.3f}, Precision: {prec:.3f}")

Оптимальный порог: 0.073
Recall: 0.890, Precision: 0.130


In [21]:
# Предсказание вероятностей для теста
probs_test = logreg.predict_proba(Xtest)[:, 1]

# Перевод в метки по найденному порогу
ytest_preds = (probs_test >= thr).astype(int)

# Метрики
print("На тестовой выборке:")
print("Матрица ошибок:\n", confusion_matrix(ytest, ytest_preds))
print("\nОтчёт по классификации:\n", classification_report(ytest, ytest_preds, digits=3))

На тестовой выборке:
Матрица ошибок:
 [[1061 2947]
 [  88  471]]

Отчёт по классификации:
               precision    recall  f1-score   support

           0      0.923     0.265     0.411      4008
           1      0.138     0.843     0.237       559

    accuracy                          0.335      4567
   macro avg      0.531     0.554     0.324      4567
weighted avg      0.827     0.335     0.390      4567



Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

Точность снизилась, но recall значительно улучшился. 

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [25]:
regs = []
recalls = []
thresholds = []
precisions = []

# Подбор по значениям C
for reg in np.arange(0.001, 1, 0.01):
    # Обучаем модель с C = reg
    model = LogisticRegression(C=reg, random_state=42, max_iter=1000)
    model.fit(Xtrain, ytrain)

    # Предсказание вероятностей на валидационной выборке
    probs_val = model.predict_proba(Xval)[:, 1]

    max_recall = -1
    thr = -1
    prec = -1

    # Перебор порогов
    for threshold in np.arange(0.05, 0.25, 0.001):
        preds = (probs_val >= threshold).astype(int)

        precision = precision_score(yval, preds, zero_division=0)
        recall = recall_score(yval, preds)

        # Условие: precision >= 0.13
        if precision >= 0.13 and recall > max_recall:
            max_recall = recall
            thr = threshold
            prec = precision

    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(reg)

In [26]:

regs = np.array(regs)
recalls = np.array(recalls)
thresholds = np.array(thresholds)
precisions = np.array(precisions)


best_idx = np.argmax(recalls)


best_C = regs[best_idx]
best_recall = recalls[best_idx]
best_precision = precisions[best_idx]
best_threshold = thresholds[best_idx]


print(f"Наилучшая модель:")
print(f"C = {best_C:.3f}")
print(f"Порог вероятности = {best_threshold:.3f}")
print(f"Recall = {best_recall:.3f}")
print(f"Precision = {best_precision:.3f}")

Наилучшая модель:
C = 0.101
Порог вероятности = 0.068
Recall = 0.963
Precision = 0.130


In [29]:
final_model = LogisticRegression(C=best_C, random_state=42, max_iter=50000)
final_model.fit(Xtrain, ytrain)

probs_test = final_model.predict_proba(Xtest)[:, 1]

ytest_preds = (probs_test >= best_threshold).astype(int)

print("📊 Матрица ошибок на тестовой выборке:")
print(confusion_matrix(ytest, ytest_preds))

print("\n📄 Отчёт по классификации на тестовой выборке:")
print(classification_report(ytest, ytest_preds, digits=3))

📊 Матрица ошибок на тестовой выборке:
[[ 722 3286]
 [  42  517]]

📄 Отчёт по классификации на тестовой выборке:
              precision    recall  f1-score   support

           0      0.945     0.180     0.303      4008
           1      0.136     0.925     0.237       559

    accuracy                          0.271      4567
   macro avg      0.540     0.553     0.270      4567
weighted avg      0.846     0.271     0.295      4567



Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

In [ ]:
Да, изменение гиперпараметра C влияет на качество модели, в пределах 15% по метрике recall

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


In [ ]:
Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче?
Да, удалось. Метод KNN давал recall около 0.011 при precision 0.188, а логистическая регрессия с оптимизированным C и порогом достигла recall
более 0.9 при precision ≈ 0.13, что в разы лучше по полноте — главной метрике в задаче.

Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?
Да, модель достигла высокого значения recall при условии, что precision не опускается ниже 0.13. Это соответствует поставленным бизнес-целям: находить как можно больше откликнувшихся клиентов, даже ценой появления ложных срабатываний.